In [ ]:
import os
import sys
import subprocess
import time

# ============================================================================ 
# 1. Configuration
# ============================================================================ 
PROJECT_ROOT = "."  # adjust if the script is not at repo root
TRAIN_SCRIPT = os.path.join(PROJECT_ROOT, "scripts", "train.py")
DATA_FILE = os.path.join(PROJECT_ROOT, "processed_data", "training_data_lincs2020_chemfiltered_1201.pkl")
OUTPUT_DIR = os.path.join(PROJECT_ROOT, "models", "single_run_lr0.0130_cosine_t0_50_gradclip1.0")

# Training params (optimized configuration)
FOLD_ID = 0
EPOCHS = 100
BATCH_SIZE = 256

# Learning rate options:
# Option 1 (Original, may cause NaN): LEARNING_RATE = 0.0139
# Option 2 (Conservative, recommended): LEARNING_RATE = 0.007
LEARNING_RATE = 0.0130  # Reduced from 0.0139 to improve stability
MIN_LR = 1e-5

DROPOUT = 0.64  # Note: This is handled in the model definition, not as CLI arg
FOCAL_GAMMA = 2.0
WEIGHT_DECAY = 1e-5
T0 = 20  # CosineAnnealing T_0 parameter
GRAD_CLIP = 1.0  # Gradient clipping to prevent explosion (NEW)
DEVICE = "auto"   # set to "cuda" to force GPU, or "cpu" otherwise
NUM_WORKERS = 4
SAVE_EVERY = 10

# ============================================================================ 
# 2. Helpers
# ============================================================================ 
def run_command(cmd_args, log_file_path):
    """Run a subprocess, stream stdout to console, and write to log file."""
    with open(log_file_path, "w", encoding="utf-8") as log_file:
        process = subprocess.Popen(
            cmd_args,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            universal_newlines=True,
            bufsize=1,
        )
        for line in process.stdout:
            print(line, end="")
            log_file.write(line)
        process.wait()
        return process.returncode

# ============================================================================ 
# 3. Main
# ============================================================================ 
print("==================================================")
print("开始单次训练 (Windows 本地版)")
print(f"训练脚本: {TRAIN_SCRIPT}")
print(f"数据文件: {DATA_FILE}")
print(f"输出目录: {OUTPUT_DIR}")
print("==================================================\n")

if not os.path.exists(TRAIN_SCRIPT):
    print(f"警告: 找不到训练脚本: {TRAIN_SCRIPT}")
    print(f"提示: 请确认训练脚本位于 scripts/train.py")
if not os.path.exists(DATA_FILE):
    print(f"警告: 找不到数据文件: {DATA_FILE}")

os.makedirs(OUTPUT_DIR, exist_ok=True)

# Build command with parameters matching the new train.py
cmd = [
    sys.executable, "-u", TRAIN_SCRIPT,
    "--data-file", DATA_FILE,
    "--output-dir", OUTPUT_DIR,
    "--fold", str(FOLD_ID),
    "--epochs", str(EPOCHS),
    "--batch-size", str(BATCH_SIZE),
    "--learning-rate", str(LEARNING_RATE),
    "--min-lr", str(MIN_LR),
    "--focal-gamma", str(FOCAL_GAMMA),
    "--weight-decay", str(WEIGHT_DECAY),
    "--t0", str(T0),
    "--grad-clip", str(GRAD_CLIP),
    "--device", DEVICE,
    "--num-workers", str(NUM_WORKERS),
    "--save-every", str(SAVE_EVERY),
]

start_time = time.time()
train_log = os.path.join(OUTPUT_DIR, "train_log.txt")
print(f"开始训练... (日志: {train_log})")
print(f"命令: {' '.join(cmd)}\n")

return_code = run_command(cmd, train_log)
elapsed = time.time() - start_time

# Save configuration summary
with open(os.path.join(OUTPUT_DIR, "config.txt"), "w", encoding="utf-8") as f:
    f.write("Single fold training configuration\n")
    f.write("="*50 + "\n")
    f.write(f"Status: {'Success' if return_code == 0 else 'Failed'}\n")
    f.write(f"Elapsed time: {elapsed:.2f}s ({elapsed/60:.1f} min)\n\n")
    f.write("Hyperparameters:\n")
    f.write(f"  Learning rate: {LEARNING_RATE}\n")
    f.write(f"  Min LR: {MIN_LR}\n")
    f.write(f"  Dropout: {DROPOUT} (fixed in model)\n")
    f.write(f"  Weight decay: {WEIGHT_DECAY}\n")
    f.write(f"  Focal gamma: {FOCAL_GAMMA}\n")
    f.write(f"  T0 (scheduler): {T0}\n")
    f.write(f"  Gradient clipping: {GRAD_CLIP}\n")
    f.write(f"  Batch size: {BATCH_SIZE}\n")
    f.write(f"  Epochs: {EPOCHS}\n")
    f.write(f"  Fold: {FOLD_ID}\n")

if return_code == 0:
    print(f"\n{'='*80}")
    print(f"✓ 训练完成！(耗时: {elapsed/60:.1f} 分钟)")
    print(f"{'='*80}")
    print(f"\n📁 生成的文件：")
    print(f"  • {OUTPUT_DIR}/model_fold_{FOLD_ID}.pt - 最终训练模型")
    print(f"  • {OUTPUT_DIR}/training_history_fold_{FOLD_ID}.pkl - 训练历史")
    print(f"  • {OUTPUT_DIR}/training_curves_fold_{FOLD_ID}.png - 训练曲线")
    print(f"  • {OUTPUT_DIR}/checkpoint_epoch_*.pt - 周期性检查点")
    print(f"  • {OUTPUT_DIR}/config.txt - 配置摘要")
    print(f"  • {OUTPUT_DIR}/train_log.txt - 完整日志\n")
    
    print(f"🎯 下一步：")
    print(f"  1. 查看训练曲线: {OUTPUT_DIR}/training_curves_fold_{FOLD_ID}.png")
    print(f"  2. 检查训练历史: {OUTPUT_DIR}/training_history_fold_{FOLD_ID}.pkl")
    print(f"  3. 加载模型进行推理")
else:
    print(f"\n{'='*80}")
    print(f"✗ 训练失败 (返回码: {return_code})")
    print(f"{'='*80}")
    print(f"请检查日志: {train_log}\n")

print("任务结束。")

开始单次训练 (Windows 本地版)
训练脚本: .\scripts\train.py
数据文件: .\processed_data\training_data_lincs2020_chemfiltered_1201.pkl
输出目录: .\models\single_run_lr0.0130_cosine_t0_50_gradclip1.0

开始训练... (日志: .\models\single_run_lr0.0130_cosine_t0_50_gradclip1.0\train_log.txt)
命令: d:\ProgramData\anaconda3\envs\chemprop\python.exe -u .\scripts\train.py --data-file .\processed_data\training_data_lincs2020_chemfiltered_1201.pkl --output-dir .\models\single_run_lr0.0130_cosine_t0_50_gradclip1.0 --fold 0 --epochs 50 --batch-size 256 --learning-rate 0.013 --min-lr 1e-05 --focal-gamma 2.0 --weight-decay 1e-05 --t0 50 --grad-clip 1.0 --device auto --num-workers 4 --save-every 10


🧬 DRUGREFLECTOR - SINGLE FOLD TRAINING (NaN-Safe Version)

📋 Configuration:
  Data file: .\processed_data\training_data_lincs2020_chemfiltered_1201.pkl
  Output dir: .\models\single_run_lr0.0130_cosine_t0_50_gradclip1.0
  Fold: 0
  Epochs: 50
  Batch size: 256
  Initial LR: 0.013
  Focal γ: 2.0
  Gradient Clip: 1.0
  Device: auto

📂 Loa